In [1]:
import pandas as pd 
import torch 
import numpy as np
from torch_geometric.data import Data, HeteroData
from torch_geometric.typing import OptTensor
from torch_scatter import scatter

graph = torch.load('sample_data/batch.pth')
from torch_geometric.utils import degree

In [2]:
def to_adj_nodes_with_times(data):
    num_nodes = data.num_nodes
    timestamps = torch.zeros((data.edge_index.shape[1], 1)) if data.timestamps is None else data.timestamps.reshape((-1,1))
    edges = torch.cat((data.edge_index.T, timestamps), dim=1) if not isinstance(data, HeteroData) else torch.cat((data['node', 'to', 'node'].edge_index.T, timestamps), dim=1)
    adj_list_out = dict([(i, []) for i in range(num_nodes)])
    adj_list_in = dict([(i, []) for i in range(num_nodes)])
    for u,v,t in edges:
        u,v,t = int(u), int(v), int(t)
        adj_list_out[u] += [(v, t)]
        adj_list_in[v] += [(u, t)]
    return adj_list_in, adj_list_out


def ports(edge_index, adj_list):
    ports = torch.zeros(edge_index.shape[1], 1)
    ports_dict = {}
    for v, nbs in adj_list.items():
        if len(nbs) < 1: continue
        a = np.array(nbs)
        a = a[a[:, -1].argsort()]
        _, idx = np.unique(a[:,[0]],return_index=True,axis=0)
        nbs_unique = a[np.sort(idx)][:,0]
        for i, u in enumerate(nbs_unique):
            ports_dict[(u,v)] = i
    for i, e in enumerate(edge_index.T):
        ports[i] = ports_dict[tuple(e.numpy())]
    return ports

In [7]:
edge_index = graph['node', 'to', 'node'].edge_index 
df = pd.DataFrame(torch.cat([edge_index.T, graph.timestamps.reshape((-1,1))], dim=1).numpy().astype('int'), columns=['source', 'target', 't'])
df = df.sort_values(['target', 't'])

# df = df.groupby('target').apply(lambda x: x.sort_values('t'))


In [9]:
%%timeit 

df[['source', 'target']].to_numpy()

1.44 ms ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
%%timeit

prev_v = -1
mapping = {}
ports = torch.zeros(graph['node', 'to', 'node'].edge_index.shape[1], 1)

j = 0
array = df[['source', 'target']].to_numpy()
for u, v in array:
    if v != prev_v:
        counter = 0
        mapping = {}
        mapping[u] = counter
        ports[j] = counter
    # else:
    #     if u in mapping:
    #         ports[j] = mapping[u]
    #     else:
    #         counter += 1
    #         mapping[u] = counter
    #         ports[j] = counter
    
    prev_v = v
    j+=1

235 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
mapping

In [ ]:
ports_pd()

In [ ]:
df[:10]

In [ ]:
df[:10]

In [ ]:
edge_index = graph['node', 'to', 'node'].edge_index 
df = pd.DataFrame(torch.cat([edge_index.T, graph.timestamps.reshape((-1,1))], dim=1).numpy().astype('int'), columns=['source', 'target', 't'])
df = df.sort_values(['target', 'source'])

In [ ]:
df

In [ ]:
%%timeit
df = new_adj(graph)

In [ ]:
from collections import defaultdict

def to_adj_nodes_with_times_2(data):
    num_nodes = data.num_nodes
    timestamps = torch.zeros((data.edge_index.shape[1], 1)) if data.timestamps is None else data.timestamps.reshape((-1,1))
    edges = torch.cat((data.edge_index.T, timestamps), dim=1) if not isinstance(data, HeteroData) else torch.cat((data['node', 'to', 'node'].edge_index.T, timestamps), dim=1)
    adj_list_out = defaultdict(list)
    adj_list_in  = defaultdict(list)
    for u,v,t in edges:
        u,v,t = int(u), int(v), int(t)
        adj_list_out[u].append((v, t))
        adj_list_out[v].append((u, t))
    return adj_list_in, adj_list_out

In [ ]:
%%timeit
adj_list_in, adj_list_out = to_adj_nodes_with_times_2(graph)


In [ ]:
%%timeit
in_ports = ports(graph['node', 'to', 'node'].edge_index, adj_list_in)

In [ ]:
%%timeit
adj_list_in, adj_list_out = to_adj_nodes_with_times(graph)

in_ports = ports(graph['node', 'to', 'node'].edge_index, adj_list_in)